In [6]:
### Creating single agent #####
import os
import asyncio
from openai import AsyncOpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display
from agents import Agent, Runner, function_tool, Trace, input_guardrail, output_guardrail, FunctionTool, GuardrailFunctionOutput, OpenAIChatCompletionsModel
from typing import Dict
from pydantic import BaseModel
import os

# STEP 1: Load environment variables
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    raise ValueError("❌ GOOGLE_API_KEY not found! Please add it to your .env file.")

base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
model_name = "gemini-2.0-flash"

# Initialize Gemini client
gemini_client = AsyncOpenAI(api_key=google_api_key, base_url=base_url)

# STEP 2: Define agent prompts
cricket_system_prompt = (
    "You are a cricket expert. You know everything about players, records, scores, "
    "and historical matches. Answer only cricket-related questions. "
    "If the user question is about a subject apart from cricket, "
    "you should NOT answer and say you are not aware. Only focus on cricket topics."
)

bollywood_system_prompt = (
    "You are a Bollywood expert. You know about actors, directors, blockbusters, "
    "and recent films. Answer only Bollywood-related questions. "
    "If the user question is about a subject apart from Bollywood, "
    "you should NOT answer and reply — I am not aware. "
    "Only focus on Bollywood-related questions."
)

# STEP 3: Ask a test question
messages = "Who is Akshay Kumar?"

# FIX: include cricket_system_prompt as system role so it doesn't answer non-cricket queries
response = await gemini_client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "system", "content": cricket_system_prompt},
        {"role": "user", "content": messages}
    ]
)

answer = response.choices[0].message.content
display(Markdown(answer))

gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

competitors = []
answers = []
competitors.append(model_name)
answers.append(answer)

CricAgent = Agent(name="Cricket genuine agent", instructions=cricket_system_prompt, model=gemini_model)

response_2 = await Runner.run(CricAgent, messages)
result = response_2

display(Markdown(f"## --Cricket Fanatic Channel--\n\n### {result.final_output}"))


I am not aware.


## --Cricket Fanatic Channel--

### I am not aware of this person. I only answer cricket-related questions.


In [3]:
## Creating two agents and run them ###

import os
import asyncio
from openai import AsyncOpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display
from agents import Agent, Runner, function_tool, Trace, input_guardrail, output_guardrail, FunctionTool, GuardrailFunctionOutput, OpenAIChatCompletionsModel
from typing import Dict
from pydantic import BaseModel
import os

# STEP 1: Load environment variables
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    raise ValueError("❌ GOOGLE_API_KEY not found! Please add it to your .env file.")

base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
model_name = "gemini-2.0-flash"

# Initialize Gemini client
gemini_client = AsyncOpenAI(api_key=google_api_key, base_url=base_url)

# STEP 2: Define agent prompts
cricket_system_prompt = (
    "You are a cricket expert. You know everything about players, records, scores, "
    "and historical matches. Answer only cricket-related questions. "
    "If the user question is about a subject apart from cricket, "
    "you should NOT answer and say you are not aware. Only focus on cricket topics."
)

bollywood_system_prompt = (
    "You are a Bollywood expert. You know about actors, directors, blockbusters, "
    "and recent films. Answer only Bollywood-related questions. "
    "If the user question is about a subject apart from Bollywood, "
    "you should NOT answer and reply — I am not aware. "
    "Only focus on Bollywood-related questions."
)

# STEP 3: Ask a test question
messages = "Who is Akshay Kumar?"

# FIX: include cricket_system_prompt as system role so it doesn't answer non-cricket queries
response = await gemini_client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "system", "content": cricket_system_prompt},
        {"role": "user", "content": messages}
    ]
)

answer = response.choices[0].message.content
#display(Markdown(answer))

gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

competitors = []
answers = []
competitors.append(model_name)
answers.append(answer)

CricAgent = Agent(name="Cricket genuine agent", instructions=cricket_system_prompt, model=gemini_model)
BollyAgent = Agent(name="Bollywood genuine agent", instructions=bollywood_system_prompt, model=gemini_model)

response = await Runner.run(CricAgent, messages)
result = response

response_2 = await Runner.run(BollyAgent, messages)
result_1 = response_2

display(Markdown(f"## --Cricket Fanatic Channel--\n\n### {result.final_output}"))
display(Markdown(f"## --Bollywood Fanatic Channel--\n\n### {result_1.final_output}"))


## --Cricket Fanatic Channel--

### I am not aware.


## --Bollywood Fanatic Channel--

### Akshay Kumar is a highly successful Indian actor, producer, and television personality who primarily works in Bollywood films. He's known for his action, comedy, and patriotic roles, and has starred in over 100 films throughout his career. Some of his notable films include "Hera Pheri," "Namastey London," "Singh Is Kinng," "Housefull," "Special 26," "Airlift," "Toilet: Ek Prem Katha," "Kesari," "Mission Mangal," and "Sooryavanshi." He is considered one of the most bankable and versatile actors in the Hindi film industry.


In [2]:
## Creating two agents and run them + BestPicker Agent (dynamic question) ##

import os
import asyncio
from openai import AsyncOpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display
from agents import Agent, Runner, function_tool, Trace, input_guardrail, output_guardrail, FunctionTool, GuardrailFunctionOutput, OpenAIChatCompletionsModel
from typing import Dict
from pydantic import BaseModel
import os

# STEP 1: Load environment variables
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    raise ValueError("❌ GOOGLE_API_KEY not found! Please add it to your .env file.")

base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
model_name = "gemini-2.0-flash"

# Initialize Gemini client
gemini_client = AsyncOpenAI(api_key=google_api_key, base_url=base_url)

# STEP 2: Define agent prompts
cricket_system_prompt = (
    "You are a cricket expert. You know everything about players, records, scores, "
    "and historical matches. Answer only cricket-related questions. "
    "If the user question is about a subject apart from cricket, "
    "you should NOT answer and say you are not aware. Only focus on cricket topics."
)

bollywood_system_prompt = (
    "You are a Bollywood expert. You know about actors, directors, blockbusters, "
    "and recent films. Answer only Bollywood-related questions. "
    "If the user question is about a subject apart from Bollywood, "
    "you should NOT answer and reply — I am not aware. "
    "Only focus on Bollywood-related questions."
)

bestpicker_system_prompt = (
    "You are the BestPicker Agent. You receive responses from multiple domain experts. "
    "Your task is to carefully read each agent's answer, determine which one best fits "
    "the user's question, and briefly explain why that answer was chosen. "
    "Output your decision clearly and concisely."
)

# STEP 3: Take user input dynamically
messages = input("💬 Enter your question: ").strip()

# FIX: include cricket_system_prompt as system role so it doesn't answer non-cricket queries
response = await gemini_client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "system", "content": cricket_system_prompt},
        {"role": "user", "content": messages}
    ]
)

answer = response.choices[0].message.content

# STEP 4: Create models and agents
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

CricAgent = Agent(name="Cricket genuine agent", instructions=cricket_system_prompt, model=gemini_model)
BollyAgent = Agent(name="Bollywood genuine agent", instructions=bollywood_system_prompt, model=gemini_model)

# Run Cricket and Bollywood agents
response = await Runner.run(CricAgent, messages)
result = response

response_2 = await Runner.run(BollyAgent, messages)
result_1 = response_2

# Display individual agent outputs
display(Markdown(f"## --Cricket Fanatic Channel--\n\n### {result.final_output}"))
display(Markdown(f"## --Bollywood Crazy Channel--\n\n### {result_1.final_output}"))

# STEP 5: Run BestPicker Agent
best_picker_input = f"""
User Question: {messages}

Cricket Agent Response: {result.final_output}

Bollywood Agent Response: {result_1.final_output}

Select which agent gave the better, more contextually relevant answer and justify briefly.
"""

BestPickerAgent = Agent(name="BestPicker Agent", instructions=bestpicker_system_prompt, model=gemini_model)

response_3 = await Runner.run(BestPickerAgent, best_picker_input)
best_result = response_3

# Display final BestPicker decision including the user message
display(Markdown(
    f"## 🏆 BestPicker Decision 🏆\n\n"
    f"**User Question:** {messages}\n\n"
    f"### {best_result.final_output}"
))


💬 Enter your question:  tell me the total number of centuries of Virat Kohli till date


## --Cricket Fanatic Channel--

### As of today, October 26, 2023, Virat Kohli has scored a total of 77 centuries in international cricket. This includes:

*   29 centuries in Test matches
*   50 centuries in One Day Internationals (ODIs)
*   0 century in T20 Internationals (T20Is)

## --Bollywood Crazy Channel--

### I am not aware.


## 🏆 BestPicker Decision 🏆

**User Question:** tell me the total number of centuries of Virat Kohli till date

### The Cricket Agent provided the better answer. The user asked a question about Virat Kohli's cricket statistics, which falls directly within the expertise of a Cricket Agent. The Bollywood Agent correctly stated its lack of knowledge on the subject.


In [4]:
## Creating two agents and a ToolCallingAgent that calls one of them dynamically ##

import os
import asyncio
from openai import AsyncOpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display
from agents import Agent, Runner, OpenAIChatCompletionsModel
import os

# STEP 1: Load environment variables
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    raise ValueError("❌ GOOGLE_API_KEY not found! Please add it to your .env file.")

base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
model_name = "gemini-2.0-flash"

# Initialize Gemini client
gemini_client = AsyncOpenAI(api_key=google_api_key, base_url=base_url)

# STEP 2: Define agent prompts
cricket_system_prompt = (
    "You are a cricket expert. You know everything about players, records, scores, "
    "and historical matches. Answer only cricket-related questions. "
    "If the user question is about a subject apart from cricket, "
    "you should NOT answer and say you are not aware."
)

bollywood_system_prompt = (
    "You are a Bollywood expert. You know about actors, directors, blockbusters, "
    "and recent films. Answer only Bollywood-related questions. "
    "If the user question is about a subject apart from Bollywood, "
    "you should NOT answer and reply — I am not aware."
)

toolcaller_prompt = (
    "You are the ToolCalling Agent. You will receive a user question and must decide "
    "which specialist agent (Cricket or Bollywood) should answer. "
    "If the question is about cricket, call the Cricket Agent. "
    "If it's about movies or Bollywood, call the Bollywood Agent. "
    "Clearly state which agent you are calling and then produce that agent’s final answer."
)

# STEP 3: Take user input dynamically
messages = input("💬 Enter your question: ").strip()

# STEP 4: Initialize model and agents
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

CricAgent = Agent(name="Cricket genuine agent", instructions=cricket_system_prompt, model=gemini_model)
BollyAgent = Agent(name="Bollywood genuine agent", instructions=bollywood_system_prompt, model=gemini_model)
ToolCallerAgent = Agent(name="ToolCalling Agent", instructions=toolcaller_prompt, model=gemini_model)

# STEP 5: ToolCalling agent decides which agent to use
decision_prompt = f"""
User Question: {messages}

You must reply in this exact JSON structure:
{{
  "chosen_agent": "Cricket" or "Bollywood",
  "reason": "short reason why"
}}
"""

tool_decision_response = await Runner.run(ToolCallerAgent, decision_prompt)
tool_decision = tool_decision_response.final_output

#display(Markdown(f"## 🧠 ToolCalling Agent Decision\n\n### {tool_decision}"))

# STEP 6: Parse decision textually (no external libs)
chosen_agent = "Cricket" if "Cricket" in tool_decision else "Bollywood"

# STEP 7: Call the chosen agent dynamically
if chosen_agent == "Cricket":
    response = await Runner.run(CricAgent, messages)
    final_result = response.final_output
else:
    response = await Runner.run(BollyAgent, messages)
    final_result = response.final_output

# STEP 8: Display final output flow
display(Markdown(
    f"## 🏁 Final Answer Flow\n\n"
    f"**User Question:** {messages}\n\n"
    f"**Chosen Agent:** {chosen_agent} Agent\n\n"
    f"**Agent Response:** {final_result}"
))


💬 Enter your question:  tell me something about Dia Mirza


## 🏁 Final Answer Flow

**User Question:** tell me something about Dia Mirza

**Chosen Agent:** Bollywood Agent

**Agent Response:** Dia Mirza is an Indian actress, producer, and environmentalist who primarily works in Hindi films. She won the Miss Asia Pacific 2000 title. Some of her notable films include "Rehnaa Hai Terre Dil Mein," "Parineeta," "Lage Raho Munna Bhai," and "Sanju." She is also a vocal advocate for environmental issues and is a UN Environment Goodwill Ambassador. Recently, she has been involved in projects like "Thappad" and "Bheed."


In [7]:
## Two agents + ToolCallingAgent with generic model-based guardrail ##

import os
import asyncio
from openai import AsyncOpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display
from agents import Agent, Runner, OpenAIChatCompletionsModel

# STEP 1: Load environment variables
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
if not google_api_key:
    raise ValueError("❌ GOOGLE_API_KEY not found in .env")

base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
model_name = "gemini-2.0-flash"
gemini_client = AsyncOpenAI(api_key=google_api_key, base_url=base_url)

# STEP 2: Define system prompts
cricket_system_prompt = (
    "You are a cricket expert. Only answer cricket-related questions."
)
bollywood_system_prompt = (
    "You are a Bollywood expert. Only answer Bollywood-related questions."
)
toolcaller_prompt = (
    "You are the ToolCalling Agent. Decide whether to use the Cricket or Bollywood agent."
)

# STEP 3: Generic model-based guardrail prompt
guardrail_prompt = """
You are a safety and relevance checker.
Given a user question, decide if it is SAFE and RELEVANT.
Return exactly one word:
- 'ALLOW' → if the question is appropriate and safe
- 'BLOCK' → if it involves hate, harm, violence, NSFW content, or any irrelevant/off-topic query
"""

# STEP 4: Take user input
messages = input("💬 Enter your question: ").strip()

gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

# --- Step 5: Generic safety validation using same LLM ---
guardrail_check = await gemini_client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "system", "content": guardrail_prompt},
        {"role": "user", "content": messages}
    ]
)
decision = guardrail_check.choices[0].message.content.strip().upper()

if "BLOCK" in decision:
    print("❌ Your input is out of context or unsafe. Execution stopped.")
else:
    # STEP 6: Initialize agents
    CricAgent = Agent(name="Cricket genuine agent", instructions=cricket_system_prompt, model=gemini_model)
    BollyAgent = Agent(name="Bollywood genuine agent", instructions=bollywood_system_prompt, model=gemini_model)
    ToolCallerAgent = Agent(name="ToolCalling Agent", instructions=toolcaller_prompt, model=gemini_model)

    # STEP 7: ToolCalling agent decides which expert to use
    decision_prompt = f"""
    User Question: {messages}
    Return in JSON:
    {{ "chosen_agent": "Cricket" or "Bollywood", "reason": "why" }}
    """

    tool_decision_response = await Runner.run(ToolCallerAgent, decision_prompt)
    tool_decision = tool_decision_response.final_output
    display(Markdown(f"## 🧠 ToolCalling Agent Decision\n\n### {tool_decision}"))

    chosen_agent = "Cricket" if "Cricket" in tool_decision else "Bollywood"

    # STEP 8: Run selected agent
    if chosen_agent == "Cricket":
        response = await Runner.run(CricAgent, messages)
        final_result = response.final_output
    else:
        response = await Runner.run(BollyAgent, messages)
        final_result = response.final_output

    # --- Generic safety check on model response ---
    safety_check = await gemini_client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": guardrail_prompt},
            {"role": "user", "content": final_result}
        ]
    )
    safety_flag = safety_check.choices[0].message.content.strip().upper()

    if "BLOCK" in safety_flag:
        print("⚠️ The model attempted to produce unsafe or off-topic content. Response blocked.")
    else:
        display(Markdown(
            f"## 🏁 Final Answer Flow\n\n"
            f"**User Question:** {messages}\n\n"
            f"**Chosen Agent:** {chosen_agent} Agent\n\n"
            f"**Agent Response:** {final_result}"
        ))


💬 Enter your question:  need some idea on doing bollywood movies


## 🧠 ToolCalling Agent Decision

### ```json
{
  "chosen_agent": "Bollywood",
  "reason": "The user's question is about Bollywood movies, so the Bollywood agent is the appropriate choice."
}
```


## 🏁 Final Answer Flow

**User Question:** need some idea on doing bollywood movies

**Chosen Agent:** Bollywood Agent

**Agent Response:** Okay, you want to make a Bollywood film! That's fantastic! Here's a breakdown of ideas, focusing on key elements that make Bollywood movies tick:

**1. Genre is King (or Queen!):**

*   **Romantic Drama:** This is the bread and butter. Think star-crossed lovers, family conflicts, societal pressures, and of course, epic song-and-dance sequences.
    *   **Idea:** A modern-day Romeo & Juliet set against the backdrop of a tech rivalry between two powerful Indian business families.
*   **Action-Packed Thriller:** From gritty underworld stories to patriotic spy thrillers, these are always in demand.
    *   **Idea:** A vigilante hero takes on a corrupt political system, uncovering a massive money laundering scheme. Lots of fight scenes and chases are required.
*   **Comedy:** From slapstick to witty banter, humor is a must.
    *   **Idea:** A group of bumbling friends accidentally stumble upon a treasure map and embark on a hilarious adventure across India.
*   **Historical/Period Drama:** These can be grand and visually stunning, but require meticulous research and attention to detail.
    *   **Idea:** A story about a forgotten queen who fought bravely against British rule. (Think *Manikarnika*, but with a fresh perspective).
*   **Socially Relevant Films:** Movies that tackle important issues like caste discrimination, gender inequality, or environmental concerns are gaining popularity.
    *   **Idea:** A young woman from a rural village breaks societal barriers to become a successful entrepreneur.

**2. The Power of "Masala":**

*   Bollywood is known for its "masala" mix – a blend of romance, action, comedy, drama, and music, all rolled into one. Even if your film leans heavily into one genre, consider adding elements from others.

**3. Music is Non-Negotiable:**

*   Hit songs are crucial for a Bollywood film's success. You'll need:
    *   **Romantic Ballads:** To tug at heartstrings.
    *   **Upbeat Dance Numbers:** For celebrations and high-energy moments.
    *   **Item Numbers (Optional):** Though controversial, they can be a huge draw.
    *   **Background Score:** To amplify emotions and create suspense.

**4. Star Power vs. Content:**

*   **Star Power:** Big stars draw audiences, but even they need a good script.
*   **Content:** A compelling story can make even a smaller-budget film a hit.
*   **The Sweet Spot:** Finding a balance between recognizable faces and a strong narrative is ideal.

**5. Key Elements to Consider:**

*   **Hero/Heroine:** Must be relatable (even if they're flawed) and have a strong character arc.
*   **Villain:** Needs to be formidable and have clear motivations.
*   **Supporting Cast:** Adds depth and humor to the story.
*   **Locations:** India offers a diverse range of stunning locations – from the Himalayas to the beaches of Goa.
*   **Costumes:** Reflect the characters' personalities and the film's setting.
*   **Dialogues:** Must be memorable and quotable.
*   **Ending:** Should be satisfying and leave the audience with a strong emotional impact.

**6. Some Trending Themes to Mull Over:**

*   **Patriotism (but with a fresh take):** Avoid jingoism. Explore more nuanced stories about national identity.
*   **Women-Centric Stories:** Empowering narratives about female resilience and achievement are always welcome.
*   **LGBTQ+ Representation:** Bollywood is becoming more inclusive.
*   **Mental Health:** Films that address mental health issues sensitively can be very impactful.
*   **Sports Dramas:** Inspired by real-life events always garner attention.

**7. Story Arcs**
*   Love triangle, good vs evil, underdog stories.

**How to get Started:**

1.  **Develop a Strong Story:** Invest time in crafting a compelling narrative with well-developed characters.
2.  **Write a Detailed Script:** Include dialogues, scene descriptions, and song placements.
3.  **Find a Producer:** Either self-produce or pitch your script to production houses.
4.  **Assemble Your Team:** Director, cinematographer, music composer, choreographer, editor, etc.
5.  **Cast Your Actors:** Consider both established stars and fresh talent.
6.  **Shoot Your Film:** Plan your schedule and budget carefully.
7.  **Post-Production:** Editing, sound design, music mixing, etc.
8.  **Marketing and Distribution:** Promote your film effectively and secure a distribution deal.

**Important Considerations:**

*   **Budget:** Bollywood films range from low-budget independent productions to massive blockbusters.
*   **Target Audience:** Who are you making this film for?
*   **Censor Board:** Be aware of the guidelines and restrictions of the Central Board of Film Certification (CBFC).

I'm here to help you brainstorm further!  Do you have a particular genre in mind, or any specific ideas you'd like to discuss?
